# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for A. Thaliana

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone

import matplotlib.pyplot as plt

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
ORGANISM="athaliana"
LOG_FILE = f"../logs/{ORGANISM}_amino_sugar.log"
N_THREADS = 16
IDENTITY_THRESHOLD=70
TAX_ID = 3702
OUTLIERS = ["O81775", "Q9SW07", "Q9FHH5", "Q8S8A0", "Q3E965", "Q3EAV6", "Q3E8L0"]

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="remove",
    outliers=OUTLIERS,
    verbose=True,
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...
Q84WN3,Sugar transport,Membrane;Transmembrane,Transport,SWEET17 At4g15920 dl4000c FCAALL.237,Bidirectional sugar transporter SWEET17 (AtSWE...,NaN,3702,MAEASFYIGVIGNVISVLVFLSPVETFWKIVKRRSTEEYKSLPYIC...
O04249,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,STP7 At4g02050 AGAA.1 T10M13.6,Sugar transport protein 7 (Hexose transporter 7),NaN,3702,MAGGSFGPTGVAKERAEQYQGKVTSYVIIACLVAAIGGSIFGYDIG...
Q56ZZ7,Sugar transport,Membrane;Plastid inner membrane;Transmembrane,Transport,At5g16150 T21H19.70,Plastidic glucose transporter 4 (AtpGlcT),2.A.1.1.102,3702,MQSSTYAVKGNAAFAFQRRTFSSDRSTTSTGIRFAGYKSLATTGPL...
...,...,...,...,...,...,...,...,...
Q94EI9,Sugar transport,Membrane;Transmembrane,Transport,At3g14410 MLN21.19,Probable sugar phosphate/phosphate translocato...,NaN,3702,MADRSKGFMRAEFVTYAYILLYIALSSGQIFFNKWVLSSKEINFPY...
Q8L6Z8,Sugar transport,Membrane;Transmembrane,Transport,At3g03090 T17B22.22,D-xylose-proton symporter-like 1,2.A.1.1.45,3702,MGFDPENQSISSVGQVVGDSSSGGITAEKEPLLKENHSPENYSVLA...
Q9FDZ5,Sugar transport,Membrane;Transmembrane,Antiport;Transport,URGT6 At1g34020 F12G12.16 T15K4.8,UDP-rhamnose/UDP-galactose transporter 6 (UDP-...,NaN,3702,MAPVSKADKKAAVDAAAWMFNVVTSVGIIIVNKALMATYGYSFATT...


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...
Q84WN3,Sugar transport,Membrane;Transmembrane,Transport,SWEET17 At4g15920 dl4000c FCAALL.237,Bidirectional sugar transporter SWEET17 (AtSWE...,NaN,3702,MAEASFYIGVIGNVISVLVFLSPVETFWKIVKRRSTEEYKSLPYIC...
O04249,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,STP7 At4g02050 AGAA.1 T10M13.6,Sugar transport protein 7 (Hexose transporter 7),NaN,3702,MAGGSFGPTGVAKERAEQYQGKVTSYVIIACLVAAIGGSIFGYDIG...
Q56ZZ7,Sugar transport,Membrane;Plastid inner membrane;Transmembrane,Transport,At5g16150 T21H19.70,Plastidic glucose transporter 4 (AtpGlcT),2.A.1.1.102,3702,MQSSTYAVKGNAAFAFQRRTFSSDRSTTSTGIRFAGYKSLATTGPL...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Sugar transport         84
Amino-acid transport    33
Name: labels, dtype: int64

## PAAC

In [7]:
calculate_composition_feature(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    feature_type="paac",
)

## Reading dataframes

In [8]:
df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)

# Functions

In [9]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [10]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy()
    sample_names = df_features.index.to_numpy()
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [11]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PAAC

### Stats

In [12]:
df_stats = get_feature_stats(df_paac, df_labels).head(10)
# 10 best features
display(df_stats.head(10))

display(df_stats.sort_values("corr", ascending=False))

,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
FL,0.393504,0.008660,0.004984,0.393504,0.005545,0.009884,0.005464,0.009072,0.003607
LF,0.302802,0.008633,0.004697,0.302802,0.006374,0.009521,0.005792,0.008889,0.003098
LP,-0.372488,0.005226,0.002930,0.372488,0.006960,0.004545,0.007042,0.004193,0.002849
FG,-0.141151,0.007673,0.003893,0.141151,0.008546,0.007330,0.008949,0.006237,0.002712
TA,-0.164300,0.004614,0.003540,0.164300,0.005538,0.004251,0.006263,0.003630,0.002633
VS,0.275030,0.007755,0.004163,0.275030,0.005936,0.008469,0.005792,0.008288,0.002496
PS,-0.275323,0.003287,0.003025,0.275323,0.004610,0.002767,0.004556,0.002119,0.002436
YT,-0.320678,0.001938,0.002287,0.320678,0.003103,0.001481,0.002347,0.000000,0.002347
SY,-0.290847,0.003100,0.002666,0.290847,0.004332,0.002616,0.004357,0.002033,0.002325
IV,-0.206625,0.006337,0.003726,0.206625,0.007560,0.005856,0.007767,0.005443,0.002324


,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
FL,0.393504,0.008660,0.004984,0.393504,0.005545,0.009884,0.005464,0.009072,0.003607
LF,0.302802,0.008633,0.004697,0.302802,0.006374,0.009521,0.005792,0.008889,0.003098
VS,0.275030,0.007755,0.004163,0.275030,0.005936,0.008469,0.005792,0.008288,0.002496
FG,-0.141151,0.007673,0.003893,0.141151,0.008546,0.007330,0.008949,0.006237,0.002712
TA,-0.164300,0.004614,0.003540,0.164300,0.005538,0.004251,0.006263,0.003630,0.002633
IV,-0.206625,0.006337,0.003726,0.206625,0.007560,0.005856,0.007767,0.005443,0.002324
PS,-0.275323,0.003287,0.003025,0.275323,0.004610,0.002767,0.004556,0.002119,0.002436
SY,-0.290847,0.003100,0.002666,0.290847,0.004332,0.002616,0.004357,0.002033,0.002325
YT,-0.320678,0.001938,0.002287,0.320678,0.003103,0.001481,0.002347,0.000000,0.002347
LP,-0.372488,0.005226,0.002930,0.372488,0.006960,0.004545,0.007042,0.004193,0.002849


### Independent test set

In [13]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_paac, df_labels)

### Model selection

In [14]:
for estimator in [LinearSVC(random_state=0), SVC(), GaussianNB(), KNeighborsClassifier(), RandomForestClassifier(random_state=0), SGDClassifier(random_state=0)]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(estimator, X_train, y_train, scoring="f1_macro")
    print("### ",type(estimator).__name__)
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")

###  LinearSVC
CV folds: [0.424 0.424 0.406 0.419 0.419]
Mean: 0.419
Std: 0.007
###  SVC
CV folds: [0.424 0.424 0.406 0.419 0.419]
Mean: 0.419
Std: 0.007
###  GaussianNB
CV folds: [0.729 0.842 0.708 0.862 0.518]
Mean: 0.731
Std: 0.123
###  KNeighborsClassifier
CV folds: [0.864 0.636 0.808 0.862 0.926]
Mean: 0.819
Std: 0.099
###  RandomForestClassifier
CV folds: [0.604 0.424 0.683 0.6   0.419]
Mean: 0.546
Std: 0.106
###  SGDClassifier
CV folds: [0.208 0.424 0.406 0.419 0.217]
Mean: 0.335
Std: 0.1


### Hyperparameters

In [15]:
params = {
    "kneighborsclassifier__n_neighbors": list(range(3,10)),
    "kneighborsclassifier__weights": ["uniform", "distance"],
    "kneighborsclassifier__metric": ["minkowski", "euclidean", "manhattan"],
}
gsearch = GridSearchCV(
    make_pipeline(StandardScaler(), KNeighborsClassifier()),
    param_grid=params,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_knn = gsearch.best_estimator_


{'kneighborsclassifier__metric': 'manhattan', 'kneighborsclassifier__n_neighbors': 4, 'kneighborsclassifier__weights': 'distance'}
0.9138533114395184


### Dimensionality reduction

In [16]:
gsearch = GridSearchCV(
    estimator=Pipeline(
        [
            ("scale", StandardScaler()),
            ("kbest", SelectKBest()),
            ("scale2", StandardScaler()),
            ("knn", KNeighborsClassifier()),
        ]
    ),
    param_grid={
        "knn__n_neighbors": list(range(3, 6)),
        "knn__weights": ["uniform", "distance"],
        "knn__metric": ["minkowski", "euclidean", "manhattan"],
        "kbest__k": list(range(1, 400)),
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_.round(3))
best_estimator_knn_kbest = gsearch.best_estimator_

{'kbest__k': 266, 'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'uniform'}
0.953


In [17]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 95% variance:", np.argmax(csum >= 0.95) + 1)

gsearch = GridSearchCV(
    estimator=Pipeline(
        [
            ("scale", StandardScaler()),
            ("pca", PCA()),
            ("scale2", StandardScaler()),
            ("knn", KNeighborsClassifier()),
        ]
    ),
    param_grid={
        "knn__n_neighbors": list(range(3, 6)),
        "knn__weights": ["uniform", "distance"],
        "knn__metric": ["minkowski", "euclidean", "manhattan"],
        "pca__n_components": np.linspace(0.8, 0.99, 20),
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_.round(3))
best_estimator_knn_pca = gsearch.best_estimator_


Number of components to explain 95% variance: 71
{'knn__metric': 'manhattan', 'knn__n_neighbors': 4, 'knn__weights': 'uniform', 'pca__n_components': 0.97}
0.89


### Validation

In [18]:
best_estimator = best_estimator_knn_kbest

In [19]:
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.953+-0.064


,Amino,Sugar,Macro,Weighted
precision,1.000,0.944,0.972,0.961
recall,0.857,1.000,0.929,0.958
f1-score,0.923,0.971,0.947,0.957
support,7.000,17.000,24.000,24.000


,Amino,Sugar
Amino,6,1
Sugar,0,17
